In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

True

In [2]:
import os
print(os.environ['GOOGLE_API_KEY'][:10]+'*'*10)

AIzaSyC06D**********


In [3]:
from langchain_teddynote import logging

logging.langsmith("Synthetic Data Generation")

LangSmith 추적을 시작합니다.
[프로젝트명]
Synthetic Data Generation


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
# loader = PDFPlumberLoader("data/Prompt_Tuning.pdf")
loader = PDFPlumberLoader("data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

25

In [5]:
docs[0].page_content

'도메인 특화 LLM: Mistral 7B를 활용한 금융 업무분야 파인튜닝 및 활용 방법\n\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080f\U000f080

In [6]:
import re
for doc in docs:
	text_org = doc.page_content
	clean_text = re.split(r'\U000f080f', text_org)[-1]
	doc.page_content = clean_text


In [7]:
docs[0].page_content

'\n그리고 LLM의 파인튜닝에 대하여 상세하게 알아 Human Feedback)시키는 학습과정을 거치면서 대화\n보고, 본 논문에서 다루는 금융분야에서의 언어 형으로 발전시킨 GPT-3.5모델을 적용하여 사람과\n모델 적용 영역으로 나누어 설명한다. 유사한 자연어를 생성하도록 학습된 AI 챗봇인\nChatGPT를 공개하여 출시 2개월만에 월간 이용자가\n2.1. LLM(Large Language Model)의 개요 1억명을 넘기며 많은 관심을 받았다(정천수, 2023b).\nChatGPT같은 오픈된 도메인에 대한 챗봇 대화\n최근 몇 년 동안, LLM은 NLP(자연어 처리) 분야\n에서는 ChatGPT와 LLM모델과의 Prompt로 인터\n에서 차별화된 발전을 이루고 있다. Transformer\n페이스를 하게 되는데, 이때 인터페이스를 하기 위한\n구조를 기반으로 한 BERT (Bidirectional Encoder\n어플리케이션 사이에 사용하는 매개변수를 적용\nRepresentations from Transformers), GPT (Generative\n할 때 주의해야 할 정보는 개인정보에 관한 것은\nPre-trained Transformer) 등의 모델은 대량의 텍스트\n회피해야 한다. 개발자가 동의 없이 사용자로부터\n데이터를 사전 훈련하여 다양한 자연어 태스크에\n데이터를 수집하고 사용하는 것을 방지하는 법이\n적용할 수 있는 강력한 표현을 학습하였다. 이들\n이미 있지만, 실제 생활에서 사용자는 개발자가\n모델은 텍스트의 문맥을 파악하고, 다양한 문법\n데이터를 얼마나 많이 가져오고, 해당 데이터가\n구조와 의미적 관계를 이해하는 데에 우수한 성\n어디에 있는지 알기 어렵기 때문이다(Jeong and\n과를 보여주고 있다. 생성형 AI(Generative AI)는\nJeong, 2022). 또한 ChatGPT에서 대화 시에는 질\n방대한 양의 학습된 데이터모델을 바탕으로 텍\n문이나 요청인 Prompt를 얼마나 자세하게 전달\n스트, 이미지, 

In [7]:
docs[0].metadata

{'source': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'file_path': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'page': 3,
 'total_pages': 29,
 'Author': 'MS',
 'CreationDate': "D:20240328014510+09'00'",
 'Creator': 'PScript5.dll Version 5.2.2',
 'ModDate': "D:20240328014510+09'00'",
 'Producer': 'Acrobat Distiller 9.3.2 (Windows)',
 'Title': '<C1F6B4C9C1A4BAB8203330B1C731C8A35FC5EBBABB2E687770>',
 'rgid': 'PB:379341632_AS:11431281232126100@1711621456089'}

In [8]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_ollama import OllamaEmbeddings, ChatOllama

# 데이터셋 생성기
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOllama(model="dnotitia/dna")
# llm = ChatOllama(model="llama3.2")


# 문서 임베딩
# embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# embeddings = OllamaEmbeddings(model="bge-m3")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [10]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
# LangChain의 gemini 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
generator_llm = LangchainLLMWrapper(llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)


c:\Users\최인규\Documents\RAG_practice\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 기존 Default 방식

## 지식그래프 생성

### 문서 노드 생성

In [15]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

In [ ]:

kg = KnowledgeGraph()
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )


In [13]:
kg

KnowledgeGraph(nodes: 25, relationships: 0)

### Transform 적용으로 노드 재생성 및 엣지(relationships) 생성

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms


# define your LLM and Embedding Model
# here we are using the same LLM and Embedding Model that we used to generate the testset
transformer_llm = generator_llm
embedding_model = ragas_embeddings

trans = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)


In [12]:
trans

[HeadlinesExtractor(name='HeadlinesExtractor', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x000001C6DE8353A0>, llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), merge_if_possible=True, max_token_limit=32000, tokenizer=<Encoding 'o200k_base'>, property_name='headlines', prompt=HeadlinesExtractorPrompt(instruction=Extract the most important max_num headlines from the given text that can be used to split the text into independent sections.Focus on Level 2 and Level 3 headings., examples=[(TextWithExtractionLimit(text='                Introduction\n                Overview of the topic...\n\n                Main Concepts\n                Explanation of core ideas...\n\n                Detailed Analysis\n                Techniques and methods for analysis...\n\n                Subsection: Specialized Techniques\n                Further details on specialized techniques...\n\n                Future Directions\n                Insights into upcoming trends...\n\n    

In [16]:
kg

KnowledgeGraph(nodes: 25, relationships: 0)

#### HeadlinesExtractor

In [17]:
len(trans[0].generate_execution_plan(kg=kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\1041252660.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(trans[0].generate_execution_plan(kg=kg))


23

In [18]:
from ragas.testset.transforms.extractors.llm_based import HeadlinesExtractor
a = HeadlinesExtractor(llm=generator_llm)
len(a.generate_execution_plan(kg=kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\966529865.py:3: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(a.generate_execution_plan(kg=kg))


25

In [19]:
from ragas.utils import num_tokens_from_string
num_tokens_from_string(kg.nodes[1].get_property("page_content"))

496

In [41]:
apply_transforms(kg, trans[:1])

Applying HeadlinesExtractor:   0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
kg.save("test_kg/kg_HeadlineExtractor.json")

In [19]:
load_kg = KnowledgeGraph().load("test_kg/kg_HeadlineExtractor.json")

#### HeadlinesSpliter

In [13]:
trans[1]

HeadlineSplitter(name='HeadlineSplitter', filter_nodes=<function default_filter at 0x000001C6DECAFB00>, min_tokens=500, max_tokens=1000)

In [22]:
len(trans[1].generate_execution_plan(kg=load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\3840880366.py:1: RuntimeWarning: coroutine 'Splitter.generate_execution_plan.<locals>.apply_split' was never awaited
  len(trans[1].generate_execution_plan(kg=load_kg))


25

In [23]:
load_kg.nodes[0].get_property("headlines")

['2.1. LLM(Large Language Model)의 개요',
 '최근 몇 년 동안, LLM은 NLP(자연어 처리) 분야',
 '생성형 AI(Generative AI)는',
 '또한 NLP 분야중 챗봇은 NLU(Natural Language Understanding) 기술의 발전으로',
 '이렇게 LLM과 생성형 AI는 AI의 딥러닝 안에 포지셔닝하고 있어']

In [24]:
len(load_kg.nodes[0].get_property("page_content").split())

402

In [25]:
trans[1].filter_nodes = trans[0].filter_nodes

In [ ]:
nodes=[node for node in load_kg.nodes if trans[1].filter_nodes(node)]

In [27]:
len(nodes)

23

In [96]:
trans[1]

HeadlineSplitter(name='HeadlineSplitter', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000160BC7842C0>, min_tokens=500, max_tokens=1000)

In [31]:
len(load_kg.nodes)

25

In [ ]:
from ragas.testset.transforms.splitters.headline import HeadlineSplitter
apply_transforms(load_kg, HeadlineSplitter(filter_nodes=trans[0].filter_nodes))

In [21]:
load_kg.save("test_kg/kg_HeadlineSpliter.json")

In [22]:
load_kg=KnowledgeGraph().load("test_kg/kg_HeadlineSpliter.json")

In [33]:
load_kg.nodes[0].get_property("headlines")

['2.1. LLM(Large Language Model)의 개요',
 '최근 몇 년 동안, LLM은 NLP(자연어 처리) 분야',
 '생성형 AI(Generative AI)는',
 '또한 NLP 분야중 챗봇은 NLU(Natural Language Understanding) 기술의 발전으로',
 '이렇게 LLM과 생성형 AI는 AI의 딥러닝 안에 포지셔닝하고 있어']

In [34]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 20)

In [37]:
load_kg.nodes[25].id

UUID('026ec017-0f40-4a60-b8e3-ad2f7792f9bd')

In [38]:
load_kg.relationships[0].source.id

UUID('fe875f11-1396-47e0-ba29-aaae374ba8b9')

In [39]:
load_kg.nodes[0]

Node(id: 026ec0, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines'])

In [41]:
node_doc = [n for n in load_kg.nodes if n.type == NodeType.DOCUMENT]
len(node_doc)

42

In [42]:
node_chunk = [n for n in load_kg.nodes if n.type == NodeType.CHUNK]
len(node_chunk)

13

In [43]:
print(load_kg.relationships[0].source.get_property("page_content"))


2.1.2. 파운데이션 모델(Foundation Model) 및 첫 번째 작업은 LLM이 학습될 리소스인 학습 데
LLM 사전학습(Pre-training) 이터 세트를 수집하는 것이다. 데이터는 책, 웹
생성형 AI 모델은 어떤 출력을 생성하는가에 사이트, 기사, 공개 데이터세트 등 다양한 소스에서
따라서 언어모델, 이미지 모델, 동영상 모델 등 가져올 수 있다. 유능한 LLM을 개발하기 위해
을 사용한다. 하지만 현재는 이미지와 텍스트를 사전 학습된 자료로 텍스트 데이터 세트를 사용
동시에 학습하는 멀티모달(Multi-modal) 모델들이 한다. 사전 학습된 코퍼스의 소스는 크게 일반 데이
하루가 다르게 성능과 기능이 업그레이드고 있고 터와 전문 데이터의 두 가지 유형으로 분류할 수
있으며 기초모델로 자리잡아가고 있다(정천수, 있으며 웹 페이지, 서적 및 대화 텍스트와 같은
2023d). 파운데이션 모델(Foundation Model)의 데 일반 데이터는 크고 다양하며 접근 가능한 특성
이터는 텍스트, 이미지, 음성, 정형데이터, 3D 시 으로 인해 대부분의 LLM에서 활용되며 LLM의
그널 등 구분하지 않고 학습에 이용되며 인간의 언어 모델링 및 일반화 능력을 향상시킬 수 있다.
창의력과 추론력을 포함한 일을 수행하며 이러한 또한 다국어 데이터, 과학 데이터 및 코드와 같은
기초모델은 방대한 양의 데이터를 비지도 학습 보다 전문화된 데이터 세트로 확장하여 LLM에
(Unsupervised learning)을 통해 모델을 학습시킨 특정 작업 해결 기능을 부여하는 연구도 발표되고
후 배포되어 사용자가 원하는 목적에 맞게 다운 있다(Chowdhery et al., 2023; Nijkamp et al., 2022).
스트림 작업에 대해 파인튜닝이나 문맥 내 학습 <그림 3>은 일반적인 LLM의 데이터 수집 및
(In-context learning)등과 같은 과정을 거처 완성 사전학습 절차를 보여주고 있다(Zhao et al., 2023).
되는 것이 파운데이션 

In [44]:
print(load_kg.relationships[0].target.get_property("page_content"))

2.1.2. 파운데이션 모델(Foundation Model) 및 첫 번째 작업은 LLM이 학습될 리소스인 학습 데 LLM 사전학습(Pre-training) 이터 세트를 수집하는 것이다. 데이터는 책, 웹 생성형 AI 모델은 어떤 출력을 생성하는가에 사이트, 기사, 공개 데이터세트 등 다양한 소스에서 따라서 언어모델, 이미지 모델, 동영상 모델 등 가져올 수 있다. 유능한 LLM을 개발하기 위해 을 사용한다. 하지만 현재는 이미지와 텍스트를 사전 학습된 자료로 텍스트 데이터 세트를 사용 동시에 학습하는 멀티모달(Multi-modal) 모델들이 한다. 사전 학습된 코퍼스의 소스는 크게 일반 데이 하루가 다르게 성능과 기능이 업그레이드고 있고 터와 전문 데이터의 두 가지 유형으로 분류할 수 있으며 기초모델로 자리잡아가고 있다(정천수, 있으며 웹 페이지, 서적 및 대화 텍스트와 같은 2023d). 파운데이션 모델(Foundation Model)의 데 일반 데이터는 크고 다양하며 접근 가능한 특성 이터는 텍스트, 이미지, 음성, 정형데이터, 3D 시 으로 인해 대부분의 LLM에서 활용되며 LLM의 그널 등 구분하지 않고 학습에 이용되며 인간의 언어 모델링 및 일반화 능력을 향상시킬 수 있다. 창의력과 추론력을 포함한 일을 수행하며 이러한 또한 다국어 데이터, 과학 데이터 및 코드와 같은 기초모델은 방대한 양의 데이터를 비지도 학습 보다 전문화된 데이터 세트로 확장하여 LLM에 (Unsupervised learning)을 통해 모델을 학습시킨 특정 작업 해결 기능을 부여하는 연구도 발표되고 후 배포되어 사용자가 원하는 목적에 맞게 다운 있다(Chowdhery et al., 2023; Nijkamp et al., 2022). 스트림 작업에 대해 파인튜닝이나 문맥 내 학습 <그림 3>은 일반적인 LLM의 데이터 수집 및 (In-context learning)등과 같은 과정을 거처 완성 사전학습 절차를 보여주고 있다(Zhao et al., 2023). 되는 것이 파운데이션 모

#### SummaryExtractor

In [45]:
trans[2]

SummaryExtractor(name='SummaryExtractor', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000296E0FA00E0>, llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), merge_if_possible=True, max_token_limit=32000, tokenizer=<Encoding 'o200k_base'>, property_name='summary', prompt=SummaryExtractorPrompt(instruction=Summarize the given text in less than 10 sentences., examples=[(StringIO(text='Artificial intelligence\n\nArtificial intelligence is transforming various industries by automating tasks that previously required human intelligence. From healthcare to finance, AI is being used to analyze vast amounts of data quickly and accurately. This technology is also driving innovations in areas like self-driving cars and personalized recommendations.'), StringIO(text='AI is revolutionizing industries by automating tasks, analyzing data, and driving innovations like self-driving cars and personalized recommendations.'))], language=english))

In [46]:
from ragas.testset.transforms.extractors.llm_based import SummaryExtractor, SummaryExtractorPrompt

In [47]:
len(SummaryExtractor().generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\515234645.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(SummaryExtractor().generate_execution_plan(load_kg))


55

In [48]:
len(trans[2].generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\346516686.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(trans[2].generate_execution_plan(load_kg))


40

In [49]:
print(SummaryExtractorPrompt().to_string())

Summarize the given text in less than 10 sentences.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"text": {"title": "Text", "type": "string"}}, "required": ["text"], "title": "StringIO", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "text": "Artificial intelligence\n\nArtificial intelligence is transforming various industries by automating tasks that previously required human intelligence. From healthcare to finance, AI is being used to analyze vast amounts of data quickly and accurately. This technology is also driving innovations in areas like self-driving cars and personalized recommendations."
}
Output: {
    "text": "AI is revolutionizing industries by automating tasks, analyzing data, and driving innovations like self-driving cars and personalized recommendations."
}
-----------------

In [ ]:
apply_transforms(load_kg, trans[2])

Applying SummaryExtractor:   0%|          | 0/40 [00:00<?, ?it/s]

Applying SummaryExtractor:  68%|██████▊   | 27/40 [00:08<00:04,  3.20it/s]Property 'summary' already exists in node '438a11'. Skipping!
Property 'summary' already exists in node 'dd6f45'. Skipping!
Applying SummaryExtractor:  90%|█████████ | 36/40 [00:10<00:01,  3.77it/s]Property 'summary' already exists in node '55746f'. Skipping!
Property 'summary' already exists in node '761566'. Skipping!
Applying SummaryExtractor:  98%|█████████▊| 39/40 [00:11<00:00,  3.76it/s]Property 'summary' already exists in node 'c60096'. Skipping!


In [51]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 20)

In [24]:
load_kg.save("test_kg/kg_SummaryExtractor.json")

In [25]:
load_kg=KnowledgeGraph().load("test_kg/kg_SummaryExtractor.json")

In [ ]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 20)

#### CustomNodeFilter

In [53]:
trans[3]

CustomNodeFilter(name='CustomNodeFilter', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000296DDF0E2A0>, llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), scoring_prompt=QuestionPotentialPrompt(instruction=Given a document summary and node content, score the content of the node in 1 to 5 range., examples=[], language=english), min_score=2, rubrics={'score1_description': 'The page content is irrelevant or does not align with the main themes or topics of the document summary.', 'score2_description': "The page content partially aligns with the document summary, but it includes unrelated details or lacks critical information related to the document's main themes.", 'score3_description': 'The page content generally reflects the document summary but may miss key details or lack depth in addressing the main themes.', 'score4_description': 'The page content aligns well with the document summary, covering the main themes and topics with minor gaps or minimal unrelated

In [54]:
from ragas.testset.transforms.filters import CustomNodeFilter, QuestionPotentialInput, QuestionPotentialOutput, QuestionPotentialPrompt

In [55]:
len(trans[3].generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\901010791.py:1: RuntimeWarning: coroutine 'NodeFilter.generate_execution_plan.<locals>.apply_filter' was never awaited
  len(trans[3].generate_execution_plan(load_kg))


13

In [56]:
print(trans[3].scoring_prompt.to_string(data = QuestionPotentialInput(document_summary="", node_content="", rubrics=trans[3].rubrics)))

Given a document summary and node content, score the content of the node in 1 to 5 range.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"score": {"description": "1 to 5 score", "title": "Score", "type": "integer"}}, "required": ["score"], "title": "QuestionPotentialOutput", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

-----------------------------

Now perform the same with the following input
input: {
    "document_summary": "",
    "node_content": "",
    "rubrics": {
        "score1_description": "The page content is irrelevant or does not align with the main themes or topics of the document summary.",
        "score2_description": "The page content partially aligns with the document summary, but it includes unrelated details or lacks critical information related to the document's main themes.",
        "score3_description": "The page c

In [ ]:
apply_transforms(load_kg, trans[3])

In [ ]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 20)

In [27]:
load_kg.save("test_kg/kg_CustomNodeFilter.json")

In [28]:
load_kg = KnowledgeGraph().load("test_kg/kg_CustomNodeFilter.json")

In [58]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 20)

#### Parallel Transformation(Extractor)
- EmbeddingExtractor
- ThemesExtractor
- NERExtractor

In [60]:
trans[4]

In [61]:
trans[4].transformations

[EmbeddingExtractor(name='EmbeddingExtractor', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000296E0B4D800>, property_name='summary_embedding', embed_property_name='summary', embedding_model=LangchainEmbeddingsWrapper(embeddings=OpenAIEmbeddings(...))),
 ThemesExtractor(name='ThemesExtractor', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000296E0F8FF60>, llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), merge_if_possible=True, max_token_limit=32000, tokenizer=<Encoding 'o200k_base'>, property_name='themes', prompt=ThemesAndConceptsExtractorPrompt(instruction=Extract the main themes and concepts from the given text., examples=[(TextWithExtractionLimit(text='Artificial intelligence is transforming industries by automating tasks requiring human intelligence. AI analyzes vast data quickly and accurately, driving innovations like self-driving cars and personalized recommendations.', max_num=10), ThemesAndConcepts(output=['Artificial intellig

In [59]:
len(trans[4].generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\229079015.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(trans[4].generate_execution_plan(load_kg))


66

In [62]:
from ragas.testset.transforms.extractors.llm_based import ThemesExtractor, ThemesAndConceptsExtractorPrompt, NERExtractor, NERPrompt

In [63]:
print(ThemesAndConceptsExtractorPrompt().to_string())

Extract the main themes and concepts from the given text.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"output": {"items": {"type": "string"}, "title": "Output", "type": "array"}}, "required": ["output"], "title": "ThemesAndConcepts", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "text": "Artificial intelligence is transforming industries by automating tasks requiring human intelligence. AI analyzes vast data quickly and accurately, driving innovations like self-driving cars and personalized recommendations.",
    "max_num": 10
}
Output: {
    "output": [
        "Artificial intelligence",
        "Automation",
        "Data analysis",
        "Innovation",
        "Self-driving cars",
        "Personalized recommendations"
    ]
}
-----------------------------

Now perform the same with t

In [64]:
print(NERPrompt().to_string())


Extract the named entities from the given text, limiting the output to the top entities. Ensure the number of entities does not exceed the specified maximum.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"entities": {"items": {"type": "string"}, "title": "Entities", "type": "array"}}, "required": ["entities"], "title": "NEROutput", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "text": "Elon Musk, the CEO of Tesla and SpaceX, announced plans to expand operations to new locations in Europe and Asia.\n                This expansion is expected to create thousands of jobs, particularly in cities like Berlin and Shanghai.",
    "max_num": 10
}
Output: {
    "entities": [
        "Elon Musk",
        "Tesla",
        "SpaceX",
        "Europe",
        "Asia",
        "Berlin",
        "Shanghai"

In [65]:
len(trans[4].transformations[0].generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\2979926559.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(trans[4].transformations[0].generate_execution_plan(load_kg))


40

In [66]:
len(trans[4].transformations[1].generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\4038458144.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(trans[4].transformations[1].generate_execution_plan(load_kg))


13

In [67]:
len(trans[4].transformations[2].generate_execution_plan(load_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_18468\3790453885.py:1: RuntimeWarning: coroutine 'Extractor.generate_execution_plan.<locals>.apply_extract' was never awaited
  len(trans[4].transformations[2].generate_execution_plan(load_kg))


13

In [ ]:
apply_transforms(load_kg, trans[4])

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/66 [00:00<?, ?it/s]Property 'summary_embedding' already exists in node '026ec0'. Skipping!
Property 'summary_embedding' already exists in node '55746f'. Skipping!
Property 'summary_embedding' already exists in node '533e23'. Skipping!
Property 'summary_embedding' already exists in node 'e94830'. Skipping!
Property 'summary_embedding' already exists in node 'e8e202'. Skipping!
Property 'summary_embedding' already exists in node 'c60096'. Skipping!
Property 'summary_embedding' already exists in node '717795'. Skipping!
Property 'summary_embedding' already exists in node '438a11'. Skipping!
Property 'summary_embedding' already exists in node 'dd6f45'. Skipping!
Property 'summary_embedding' already exists in node '776df5'. Skipping!
Property 'summary_embedding' already exists in node '6c5799'. Skipping!
Property 'summary_embedding' already exists in node 'c07e55'. Skipping!
Property 'summary_embedding' already

In [30]:
load_kg.save("test_kg/kg_ParallelExtractor.json")

In [31]:
load_kg = KnowledgeGraph().load("test_kg/kg_ParallelExtractor.json")

In [69]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 20)

#### Parallel Transformation 2(Relation Builder)
- CosineSimilarityBuilder
- OverlapScoreBuilder

In [70]:
trans[5]

In [71]:
trans[5].transformations

[CosineSimilarityBuilder(name='CosineSimilarityBuilder', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000296E2A93240>, property_name='summary_embedding', new_property_name='summary_similarity', threshold=0.7),
 OverlapScoreBuilder(name='', filter_nodes=<function default_transforms.<locals>.<lambda> at 0x00000296E2A93420>, property_name='entities', key_name=None, new_property_name='overlap_score', distance_measure=<DistanceMeasure.JARO_WINKLER: 'jaro_winkler'>, distance_threshold=0.9, threshold=0.01)]

In [72]:
trans[5].generate_execution_plan(load_kg)

[<coroutine object RelationshipBuilder.generate_execution_plan.<locals>.apply_build_relationships at 0x00000296DD90E420>,
 <coroutine object RelationshipBuilder.generate_execution_plan.<locals>.apply_build_relationships at 0x00000296DD90EB20>]

In [73]:
trans[5].transformations[0].generate_execution_plan(load_kg)

[<coroutine object RelationshipBuilder.generate_execution_plan.<locals>.apply_build_relationships at 0x00000296DD90EC00>]

In [74]:
trans[5].transformations[1].generate_execution_plan(load_kg)

[<coroutine object RelationshipBuilder.generate_execution_plan.<locals>.apply_build_relationships at 0x00000296DD90E6C0>]

In [ ]:
apply_transforms(load_kg, trans[5])

In [34]:
load_kg.save("test_kg/kg_RelationshipBuilder.json")

In [35]:
load_kg = KnowledgeGraph().load("test_kg/kg_RelationshipBuilder.json")

In [76]:
load_kg

KnowledgeGraph(nodes: 55, relationships: 68)

## Query Generation

### Persona

In [77]:
loaded_kg = KnowledgeGraph.load("test_kg/kg_trans6.json")
loaded_kg

KnowledgeGraph(nodes: 55, relationships: 68)

In [86]:
from ragas.testset.persona import default_filter
import numpy as np
filter_fn = default_filter
num_personas = 3
nodes = [node for node in loaded_kg.nodes if filter_fn(node)]
if len(nodes) == 0:
    raise ValueError(
        "No nodes that satisfied the given filer. Try changing the filter."
    )

summaries = [node.properties.get("summary") for node in nodes]
summaries = [summary for summary in summaries if isinstance(summary, str)]
num_personas = min(num_personas, len(summaries))

embeddings = []
for node in nodes:
    embeddings.append(node.properties.get("summary_embedding"))

embeddings = np.array(embeddings)
cosine_similarities = np.dot(embeddings, embeddings.T)

groups = []
visited = set()
threshold = 0.75

for i, _ in enumerate(summaries):
    if i in visited:
        continue
    group = [i]
    visited.add(i)
    for j in range(i + 1, len(summaries)):
        if cosine_similarities[i, j] > threshold:
            group.append(j)
            visited.add(j)
    groups.append(group)

top_summaries = []
for group in groups:
    representative_summary = max([summaries[i] for i in group], key=len)
    top_summaries.append(representative_summary)

if len(top_summaries) <= num_personas:
    top_summaries.extend(
        np.random.choice(top_summaries, num_personas - len(top_summaries))
    )

In [106]:
len(groups)

20

In [103]:
summaries[3]

"Foundation models, particularly large language models (LLMs), are trained using diverse data sources such as books, websites, and public datasets. These models are evolving rapidly, with advancements in multi-modal capabilities that allow simultaneous learning from text and images. The training data is categorized into general and specialized datasets, enhancing the model's language modeling and generalization abilities. LLMs utilize unsupervised learning techniques to process vast amounts of data, which can include multilingual and scientific data. Fine-tuning and in-context learning are essential processes for adapting these models to specific tasks. The training of LLMs requires significant computational resources, often employing model parallelization to reduce training time. Pre-trained language models (PLMs) are developed using self-supervised or semi-supervised learning methods on large, unlabeled text corpora. Overall, foundation models are becoming foundational in AI, with on

In [70]:
from ragas.testset.persona import PersonaGenerationPrompt, generate_personas_from_kg

In [104]:
print(PersonaGenerationPrompt().to_string())

Using the provided summary, generate a single persona who would likely interact with or benefit from the content. Include a unique name and a concise role description of who they are.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"name": {"title": "Name", "type": "string"}, "role_description": {"title": "Role Description", "type": "string"}}, "required": ["name", "role_description"], "title": "Persona", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "text": "Guide to Digital Marketing explains strategies for engaging audiences across various online platforms."
}
Output: {
    "name": "Digital Marketing Specialist",
    "role_description": "Focuses on engaging audiences and growing the brand online."
}
-----------------------------

Now perform the same with the following input
Input: (None)


In [108]:
persona_list=generate_personas_from_kg(
    llm = generator_llm,
    kg = loaded_kg,
    num_personas=20
    )

Generating personas:   0%|          | 0/20 [00:00<?, ?it/s]

Generating personas: 100%|██████████| 20/20 [00:05<00:00,  3.79it/s]


In [109]:
persona_list

[Persona(name='Financial Technology Analyst', role_description='Analyzes the application of large language models in the finance sector, focusing on customer service automation and AI-driven solutions.'),
 Persona(name='AI Product Manager', role_description='Oversees the development of AI products, ensuring they leverage the latest advancements in language models and remain competitive in the market.'),
 Persona(name='AI Solutions Architect', role_description='Designs and implements AI models and solutions, focusing on performance and customer satisfaction.'),
 Persona(name='AI Research Scientist', role_description='Engages in research and development of foundation models, focusing on advancements in language processing and multi-modal learning.'),
 Persona(name='AI Research Engineer', role_description='Specializes in improving language models through fine-tuning and exploring efficient methods for domain-specific adaptations.'),
 Persona(name='AI Model Optimization Engineer', role_des

### 시나리오 생성

In [78]:
from ragas.testset.synthesizers import default_query_distribution
query_distribution = default_query_distribution(generator_llm)

#### Sigle Hop

In [118]:
filtered_nodes = query_distribution[0][0].get_node_clusters(loaded_kg)

In [ ]:
# 샘플사이즈를 고려해 각 노드에서 생성할 쿼리의 수를 계산
int(np.ceil(10 / len(query_distribution[0][0].get_node_clusters(loaded_kg))))

1

In [117]:
from ragas.testset.synthesizers.prompts import ThemesPersonasMatchingPrompt, ThemesPersonasInput

In [120]:
filtered_nodes[0].get_property("entities")

['Foundation Model',
 'LLM',
 'Unsupervised learning',
 'Supervised learning',
 'Self-Supervised Learning',
 'Semi-Supervised Learning',
 'Wikipidia',
 'Chowdhery et al.',
 'Nijkamp et al.',
 'Zhao et al.']

In [121]:
persona_input = ThemesPersonasInput(themes=filtered_nodes[0].get_property("entities"), personas=persona_list)

In [122]:
persona_input

ThemesPersonasInput(themes=['Foundation Model', 'LLM', 'Unsupervised learning', 'Supervised learning', 'Self-Supervised Learning', 'Semi-Supervised Learning', 'Wikipidia', 'Chowdhery et al.', 'Nijkamp et al.', 'Zhao et al.'], personas=[Persona(name='Financial Technology Analyst', role_description='Analyzes the application of large language models in the finance sector, focusing on customer service automation and AI-driven solutions.'), Persona(name='AI Product Manager', role_description='Oversees the development of AI products, ensuring they leverage the latest advancements in language models and remain competitive in the market.'), Persona(name='AI Solutions Architect', role_description='Designs and implements AI models and solutions, focusing on performance and customer satisfaction.'), Persona(name='AI Research Scientist', role_description='Engages in research and development of foundation models, focusing on advancements in language processing and multi-modal learning.'), Persona(n

In [124]:
print(ThemesPersonasMatchingPrompt().to_string(data=persona_input))

Given a list of themes and personas with their roles, associate each persona with relevant themes based on their role description.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"mapping": {"additionalProperties": {"items": {"type": "string"}, "type": "array"}, "title": "Mapping", "type": "object"}}, "required": ["mapping"], "title": "PersonaThemesMapping", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "themes": [
        "Empathy",
        "Inclusivity",
        "Remote work"
    ],
    "personas": [
        {
            "name": "HR Manager",
            "role_description": "Focuses on inclusivity and employee support."
        },
        {
            "name": "Remote Team Lead",
            "role_description": "Manages remote team communication."
        }
    ]
}
Output: {
    "mapping":

In [127]:
persona_concepts = await ThemesPersonasMatchingPrompt().generate(data=persona_input, llm=generator_llm)

In [129]:
persona_concepts.mapping

{'Financial Technology Analyst': ['LLM',
  'Foundation Model',
  'Supervised learning'],
 'AI Product Manager': ['Foundation Model', 'LLM'],
 'AI Solutions Architect': ['Foundation Model', 'LLM'],
 'AI Research Scientist': ['Foundation Model',
  'Unsupervised learning',
  'Self-Supervised Learning'],
 'AI Research Engineer': ['LLM',
  'Supervised learning',
  'Self-Supervised Learning'],
 'AI Model Optimization Engineer': ['LLM', 'Supervised learning'],
 'Financial Data Scientist': ['LLM', 'Foundation Model'],
 'Financial Data Analyst': ['LLM', 'Supervised learning'],
 'Machine Learning Engineer': ['LLM', 'Supervised learning'],
 'Financial AI Specialist': ['LLM', 'Supervised learning'],
 'Insurance Technology Analyst': ['LLM', 'Foundation Model'],
 'AI Research Analyst': ['Foundation Model', 'LLM']}

In [130]:
base_scenarios  = query_distribution[0][0].prepare_combinations(
                filtered_nodes[0],
                filtered_nodes[0].get_property("entities"),
                personas=persona_list,
                persona_concepts=persona_concepts.mapping)

In [134]:
scenario_sample_list = query_distribution[0][0].sample_combinations(base_scenarios,1)
scenario_sample_list

[SingleHopScenario(
 nodes=1
 term=Supervised learning
 persona=name='AI Solutions Architect' role_description='Designs and implements AI models and solutions, focusing on performance and customer satisfaction.'
 style=QueryStyle.PERFECT_GRAMMAR
 length=QueryLength.SHORT)]

In [ ]:
query_distribution[0][0].generate_sample(scenario=scenario_sample_list[0])

In [135]:
scenario_sample_list[0].nodes

[Node(id: 421d36, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes'])]

In [116]:
print(ThemesPersonasMatchingPrompt().to_string())

Given a list of themes and personas with their roles, associate each persona with relevant themes based on their role description.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"mapping": {"additionalProperties": {"items": {"type": "string"}, "type": "array"}, "title": "Mapping", "type": "object"}}, "required": ["mapping"], "title": "PersonaThemesMapping", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "themes": [
        "Empathy",
        "Inclusivity",
        "Remote work"
    ],
    "personas": [
        {
            "name": "HR Manager",
            "role_description": "Focuses on inclusivity and employee support."
        },
        {
            "name": "Remote Team Lead",
            "role_description": "Manages remote team communication."
        }
    ]
}
Output: {
    "mapping":

In [79]:
len(query_distribution)

3

In [80]:
query_distribution

[(SingleHopSpecificQuerySynthesizer(name='single_hop_specifc_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:
  1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.
  2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.
  , examples=[(QueryCondition(persona=Persona(name='Software Engineer', role_description='Focuses on coding best practices and system design.'), term='microserv

In [137]:
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer
from ragas.testset.synthesizers.single_hop.prompts import QueryAnswerGenerationPrompt

In [139]:
print(QueryAnswerGenerationPrompt().to_string())

Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:
1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.
2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.

Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"query": {"title": "Query", "type": "string"}, "answer": {"title": "Answer", "type": "string"}}, "required": ["query", "answer"], "title": "GeneratedQueryAnswer", "type": "object"}Do not use single quotes in your response but double quotes,prope

In [142]:
sample = await query_distribution[0][0].generate_sample(scenario=scenario_sample_list[0])

In [144]:
sample.user_input

'How is supervised learning utilized in the training of foundation models?'

In [145]:
sample.reference

'Supervised learning is used in the training of foundation models by learning from pre-processed text data.'

In [149]:
print(sample.reference_contexts[0])


2.1.2. 파운데이션 모델(Foundation Model) 및 첫 번째 작업은 LLM이 학습될 리소스인 학습 데 LLM 사전학습(Pre-training) 이터 세트를 수집하는 것이다. 데이터는 책, 웹 생성형 AI 모델은 어떤 출력을 생성하는가에 사이트, 기사, 공개 데이터세트 등 다양한 소스에서 따라서 언어모델, 이미지 모델, 동영상 모델 등 가져올 수 있다. 유능한 LLM을 개발하기 위해 을 사용한다. 하지만 현재는 이미지와 텍스트를 사전 학습된 자료로 텍스트 데이터 세트를 사용 동시에 학습하는 멀티모달(Multi-modal) 모델들이 한다. 사전 학습된 코퍼스의 소스는 크게 일반 데이 하루가 다르게 성능과 기능이 업그레이드고 있고 터와 전문 데이터의 두 가지 유형으로 분류할 수 있으며 기초모델로 자리잡아가고 있다(정천수, 있으며 웹 페이지, 서적 및 대화 텍스트와 같은 2023d). 파운데이션 모델(Foundation Model)의 데 일반 데이터는 크고 다양하며 접근 가능한 특성 이터는 텍스트, 이미지, 음성, 정형데이터, 3D 시 으로 인해 대부분의 LLM에서 활용되며 LLM의 그널 등 구분하지 않고 학습에 이용되며 인간의 언어 모델링 및 일반화 능력을 향상시킬 수 있다. 창의력과 추론력을 포함한 일을 수행하며 이러한 또한 다국어 데이터, 과학 데이터 및 코드와 같은 기초모델은 방대한 양의 데이터를 비지도 학습 보다 전문화된 데이터 세트로 확장하여 LLM에 (Unsupervised learning)을 통해 모델을 학습시킨 특정 작업 해결 기능을 부여하는 연구도 발표되고 후 배포되어 사용자가 원하는 목적에 맞게 다운 있다(Chowdhery et al., 2023; Nijkamp et al., 2022). 스트림 작업에 대해 파인튜닝이나 문맥 내 학습 <그림 3>은 일반적인 LLM의 데이터 수집 및 (In-context learning)등과 같은 과정을 거처 완성 사전학습 절차를 보여주고 있다(Zhao et al., 2023). 되는 것이 파운데이션 모

In [21]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

for query, _ in distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [25]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loaded_kg)

In [42]:
scenario_sample_list[0]

[SingleHopScenario(
 nodes=1
 term=
 persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
 style=QueryStyle.PERFECT_GRAMMAR
 length=QueryLength.MEDIUM),
 SingleHopScenario(
 nodes=1
 term=
 persona=name='Financial Data Scientist' role_description='Specializes in analyzing financial data using advanced algorithms to ensure compliance, security, and accurate predictions.'
 style=QueryStyle.PERFECT_GRAMMAR
 length=QueryLength.SHORT)]

In [152]:
[n for n in loaded_kg.nodes if n.type == NodeType.CHUNK]

[Node(id: 421d36, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes']),
 Node(id: e4d209, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes']),
 Node(id: 8cb85b, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes']),
 Node(id: a4b129, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: 9d4361, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: 479c20, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: 3fd12d, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: e4b2f0, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: cab49c, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: 6b320f, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']),
 Node(id: 8eba31, type: NodeType.CHUNK, properties: ['page_content', '

In [154]:
[rel for rel in loaded_kg.relationships if rel.type == "entities_overlap"]

[Relationship(Node(id: a4b129) -> Node(id: e4b2f0), type: entities_overlap, properties: ['entities_overlap_score', 'overlapped_items']),
 Relationship(Node(id: a4b129) -> Node(id: d75d66), type: entities_overlap, properties: ['entities_overlap_score', 'overlapped_items']),
 Relationship(Node(id: 479c20) -> Node(id: e4b2f0), type: entities_overlap, properties: ['entities_overlap_score', 'overlapped_items']),
 Relationship(Node(id: e4b2f0) -> Node(id: cab49c), type: entities_overlap, properties: ['entities_overlap_score', 'overlapped_items'])]

In [160]:
query_distribution[1][0]

MultiHopAbstractQuerySynthesizer(name='multi_hop_abstract_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a multi-hop query and answer based on the specified conditions (persona, themes, style, length) and the provided context. The themes represent a set of phrases either extracted or generated from the context, which highlight the suitability of the selected context for multi-hop query creation. Ensure the query explicitly incorporates these themes.### Instructions:
1. **Generate a Multi-Hop Query**: Use the provided context segments and themes to form a query that requires combining information from multiple segments (e.g., `<1-hop>` and `<2-hop>`). Ensure the query explicitly incorporates one or more themes and reflects their relevance to the context.
2. **Generate an Answer**: Use only the content from the provided context to create a detailed and faithful answer to the quer

In [161]:
query_distribution[2][0]

MultiHopSpecificQuerySynthesizer(name='multi_hop_specific_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a multi-hop query and answer based on the specified conditions (persona, themes, style, length) and the provided context. The themes represent a set of phrases either extracted or generated from the context, which highlight the suitability of the selected context for multi-hop query creation. Ensure the query explicitly incorporates these themes.### Instructions:
1. **Generate a Multi-Hop Query**: Use the provided context segments and themes to form a query that requires combining information from multiple segments (e.g., `<1-hop>` and `<2-hop>`). Ensure the query explicitly incorporates one or more themes and reflects their relevance to the context.
2. **Generate an Answer**: Use only the content from the provided context to create a detailed and faithful answer to the quer

In [158]:
triplets = query_distribution[2][0].get_node_clusters(loaded_kg)

In [159]:
len(triplets)

4

#### 시나리오 생성시 한글 적용 

In [172]:
for query, _ in query_distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [173]:
testset_generator = TestsetGenerator(embedding_model=ragas_embeddings, llm=generator_llm, knowledge_graph=loaded_kg, persona_list=persona_list)
samples = testset_generator.generate(testset_size=20, query_distribution=query_distribution)

Generating Samples: 100%|██████████| 22/22 [00:07<00:00,  2.94it/s]


In [174]:
df = samples.to_pandas()

In [182]:
df

,user_input,reference_contexts,reference,synthesizer_name
0,Nijkamp et al.의 연구에서 파운데이션 모델의 특징은 무엇인가요?,[2.1.2. 파운데이션 모델(Foundation Model) 및 첫 번째 작업은 ...,"Nijkamp et al.의 연구에 따르면, 파운데이션 모델은 일반 데이터와 전문 ...",single_hop_specifc_query_synthesizer
1,2023d에서 언급된 파인튜닝의 목적은 무엇인가요?,"[사전학습하기 위한 사례에 맞게 파인튜닝을 하게 된다(정천수, 2023d). <그림...",2023d에서는 사전학습하기 위한 사례에 맞게 파인튜닝을 하게 된다고 언급하고 있습니다.,single_hop_specifc_query_synthesizer
2,금융 뉴스의 하이퍼파라미터 조정이 모델 성능에 미치는 영향은 무엇입니까?,[2) 모델의 용도: 모델의 용도에 따라 적합한 3.2.2. 하이퍼파라미터 조정 모...,"금융 뉴스 하이퍼파라미터 조정은 모델의 성능을 극대화하기 위해 중요한 단계이며, 감...",single_hop_specifc_query_synthesizer
3,BloombergGPT의 가용성에 대한 정보는 무엇인가요?,[모델의 가용성: 모델이 공개되어 있지 않다 적절히 조정함으로써 모델의 학습과 일반...,"BloombergGPT는 금융 특화 대규모 언어 모델로, 100억 개의 매개변수를 ...",single_hop_specifc_query_synthesizer
4,금융 분야에서 LLM 파인튜닝을 수행할 때 고려해야 할 사항은 무엇인가요?,[3.3.2. 파인튜닝 알고리즘 적용 이터 보안 및 개인정보 보호를 고려하여 데이터...,"금융 분야의 LLM 파인튜닝을 수행할 때는 데이터 보안, 개인정보 보호, 규정 준수...",single_hop_specifc_query_synthesizer
5,금융 분야에서 BloombergGPT의 역할은 무엇입니까?,[3.4. 금융 분야를 위한 LLM 구성 3.3.3. 보안 및 규정 준수 금융 분야...,"BloombergGPT는 금융 분야에서 자연어 처리에 특화된 모델로, 금융 데이터의...",single_hop_specifc_query_synthesizer
6,"금융 특화 파인튜닝이란 무엇이며, 이 기술이 금융 분야에서 어떻게 높은 성능을 발휘...","[하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융 특화 파인튜닝 LL...",금융 특화 파인튜닝은 금융 분야의 특정 작업에 높은 성능을 보이는 LLM(대형 언어...,single_hop_specifc_query_synthesizer
7,금융 데이터의 특성을 고려한 파인튜닝이 고객 응대 개선에 어떻게 기여할 수 있나요?,[<1-hop>\n\n금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생...,금융 데이터의 특성을 고려한 파인튜닝은 LLM 모델의 성능을 향상시켜 금융 예측 및...,multi_hop_abstract_query_synthesizer
8,금융 데이터의 특성을 고려한 LLM의 고객 응대 개선 가능성은 무엇인가요?,[<1-hop>\n\n금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생...,금융 데이터의 특성을 고려한 LLM은 고객 응대 개선에 있어 자동 응답 시스템과 금...,multi_hop_abstract_query_synthesizer
9,금융 데이터의 특성을 고려한 LLM의 파인튜닝이 고객 응대 개선에 어떻게 기여할 수...,[<1-hop>\n\n금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생...,금융 데이터의 특성을 고려한 LLM의 파인튜닝은 고객 응대 개선에 기여할 수 있는 ...,multi_hop_abstract_query_synthesizer


In [169]:
for q in df.user_input:
    print(q)

What role does Semi-Supervised Learning play in the development of foundation models?
What does the term '2023d' refer to in the context of fine-tuning for pre-training?
금융 특화 LLM의 하이퍼파라미터 조정은 왜 중요한가?
Mistral-7B는 어떤 모델이고, 금융 분야에서 어떻게 사용될 수 있나요?
금융 데이터의 특성을 고려한 파인튜닝이 고객 응대 개선에 어떻게 기여할 수 있는지 설명해줘.
What are the implications of fine-tuning LLMs for financial data in improving customer service and predictive accuracy?
What are the implications of fine-tuning LLMs for financial data in improving customer service and predictive accuracy?
금융 분야 특화 모델인 BloombergGPT와 FinBERT의 성능 극대화에 필요한 하이퍼파라미터 조정은 어떻게 이루어져야 하나요?
GPT-4와 FinGPT는 금융 분야에서 어떻게 활용되고 있으며, 이 두 모델의 차이점은 무엇인가요?
FinGPT와 BloombergGPT는 금융 분야에서 어떻게 특화된 파인튜닝을 통해 성능을 향상시키고 있나요?
What advancements have been made in financial language models like BloombergGPT and how do they compare to other models such as FinGPT and BLOOM in terms of their training and application in the finance sector?
Mistral-7B SLM은 금융 데이터의 특성을 고려한 파인튜닝을 통해 어떤 성능 향상을 이루었나요?


In [171]:
df

,user_input,reference_contexts,reference,synthesizer_name
0,What role does Semi-Supervised Learning play i...,[2.1.2. 파운데이션 모델(Foundation Model) 및 첫 번째 작업은 ...,Semi-Supervised Learning is utilized in the de...,single_hop_specifc_query_synthesizer
1,What does the term '2023d' refer to in the con...,"[사전학습하기 위한 사례에 맞게 파인튜닝을 하게 된다(정천수, 2023d). <그림...",'2023d' refers to a source by Jeong Cheon-soo ...,single_hop_specifc_query_synthesizer
2,금융 특화 LLM의 하이퍼파라미터 조정은 왜 중요한가?,[2) 모델의 용도: 모델의 용도에 따라 적합한 3.2.2. 하이퍼파라미터 조정 모...,금융 특화 LLM의 하이퍼파라미터 조정은 모델의 성능을 극대화하기 위해 중요한 단계...,single_hop_specifc_query_synthesizer
3,"Mistral-7B는 어떤 모델이고, 금융 분야에서 어떻게 사용될 수 있나요?",[모델의 가용성: 모델이 공개되어 있지 않다 적절히 조정함으로써 모델의 학습과 일반...,Mistral-7B는 프랑스의 스타트업 미스트랄 AI가 개발한 매개변수 73억 개의...,single_hop_specifc_query_synthesizer
4,금융 데이터의 특성을 고려한 파인튜닝이 고객 응대 개선에 어떻게 기여할 수 있는지 ...,[<1-hop>\n\n금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생...,금융 데이터의 특성을 고려한 파인튜닝은 고객 응대 개선에 기여할 수 있는 여러 방법...,multi_hop_abstract_query_synthesizer
5,What are the implications of fine-tuning LLMs ...,[<1-hop>\n\n금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생...,Fine-tuning LLMs for financial data has signif...,multi_hop_abstract_query_synthesizer
6,What are the implications of fine-tuning LLMs ...,[<1-hop>\n\n금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생...,Fine-tuning LLMs for financial data has signif...,multi_hop_abstract_query_synthesizer
7,금융 분야 특화 모델인 BloombergGPT와 FinBERT의 성능 극대화에 필요...,[<1-hop>\n\n2) 모델의 용도: 모델의 용도에 따라 적합한 3.2.2. 하...,금융 분야 특화 모델인 BloombergGPT와 FinBERT의 성능 극대화를 위해...,multi_hop_abstract_query_synthesizer
8,"GPT-4와 FinGPT는 금융 분야에서 어떻게 활용되고 있으며, 이 두 모델의 차...",[<1-hop>\n\n모델의 가용성: 모델이 공개되어 있지 않다 적절히 조정함으로써...,"GPT-4는 OpenAI에서 개발한 대규모 언어 모델로, 1,750억 개 이상의 매...",multi_hop_specific_query_synthesizer
9,FinGPT와 BloombergGPT는 금융 분야에서 어떻게 특화된 파인튜닝을 통해...,[<1-hop>\n\n3.4. 금융 분야를 위한 LLM 구성 3.3.3. 보안 및 ...,FinGPT와 BloombergGPT는 금융 분야에 특화된 파인튜닝을 통해 성능을 ...,multi_hop_specific_query_synthesizer


# 중복 Node 제거 테스트

## 1. HeadlineSpliter 적용이후

In [306]:
loaded_kg=KnowledgeGraph().load("test_kg/kg_HeadlineSpliter.json")

In [307]:
loaded_kg

KnowledgeGraph(nodes: 55, relationships: 20)

In [42]:
unique_nodes = set()
for node in loaded_kg.nodes:
    unique_nodes.add(node)


In [44]:
len(unique_nodes)

38

In [48]:
unique_rels = set()
for rel in loaded_kg.relationships:
    unique_rels.add(rel)

In [49]:
len(unique_rels)

20

In [45]:
for rel in loaded_kg.relationships:
    if (rel.source in unique_nodes) & (rel.target in unique_nodes):
        continue
    else:
        print(rel.source, rel.target)

In [300]:
from ragas.testset.transforms.base import NodeFilter
# from collections import defaultdict

In [319]:
check_node = loaded_kg.nodes[0]

In [325]:
[[idx for idx, node in enumerate(loaded_kg.nodes) if node == check_node] for check_node in loaded_kg.nodes]

[[0, 25],
 [1],
 [2, 26],
 [3, 27],
 [4],
 [5, 30],
 [6, 31],
 [7, 32],
 [8, 33],
 [9, 34],
 [10],
 [11, 37],
 [12, 38],
 [13],
 [14],
 [15, 44],
 [16, 45],
 [17],
 [18, 46],
 [19, 47],
 [20],
 [21],
 [22, 52],
 [23, 53],
 [24, 54],
 [0, 25],
 [2, 26],
 [3, 27],
 [28],
 [29],
 [5, 30],
 [6, 31],
 [7, 32],
 [8, 33],
 [9, 34],
 [35],
 [36],
 [11, 37],
 [12, 38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [15, 44],
 [16, 45],
 [18, 46],
 [19, 47],
 [48],
 [49],
 [50],
 [51],
 [22, 52],
 [23, 53],
 [24, 54]]

In [301]:
class DuplicateFilter(NodeFilter):
    async def custom_filter(self, node, kg):
        return len([n for n in kg.nodes if n==node])>1

In [302]:
len(DuplicateFilter().generate_execution_plan(loaded_kg))

C:\Users\최인규\AppData\Local\Temp\ipykernel_14412\1716648303.py:1: RuntimeWarning: coroutine 'NodeFilter.generate_execution_plan.<locals>.apply_filter' was never awaited
  len(DuplicateFilter().generate_execution_plan(loaded_kg))


55

In [303]:
loaded_kg

KnowledgeGraph(nodes: 55, relationships: 20)

In [330]:
from copy import deepcopy
test_kg = deepcopy(loaded_kg)
apply_transforms(test_kg, DuplicateFilter())

In [331]:
[n for n in loaded_kg.nodes if n not in test_kg.nodes]

[]

In [50]:
new_kg = KnowledgeGraph(nodes=list(unique_nodes), relationships=loaded_kg.relationships)

In [51]:
new_kg.save("test_kg/kg_HeadlineSpliter_NoDupli.json")

In [326]:
new_kg=KnowledgeGraph().load("test_kg/kg_HeadlineSpliter_nodupli.json")

In [55]:
new_kg

KnowledgeGraph(nodes: 38, relationships: 20)

## 이후 변환 적용

In [56]:
apply_transforms(new_kg,trans[2:])

Applying SummaryExtractor:   0%|          | 0/23 [00:00<?, ?it/s]

In [57]:
new_kg

KnowledgeGraph(nodes: 38, relationships: 32)

In [58]:
new_kg.nodes

[Node(id: d5f1ac, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']),
 Node(id: dd6f45, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']),
 Node(id: 717795, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']),
 Node(id: e8e202, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']),
 Node(id: cab49c, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes']),
 Node(id: 6c5799, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']),
 Node(id: c07e55, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']),
 Node(id: 3fd12d, type: NodeType.CHUNK, properties: ['page_content'

In [59]:
new_kg.save("test_kg/kg_RelationBuilder_NoDupli.json")

In [60]:
new_kg=KnowledgeGraph().load("test_kg/kg_RelationBuilder_NoDupli.json")

## Query 생성

In [61]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=new_kg)

In [62]:
from ragas.testset.synthesizers import default_query_distribution
query_distribution = default_query_distribution(generator_llm)

In [63]:
query_distribution

[(SingleHopSpecificQuerySynthesizer(name='single_hop_specifc_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:
  1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.
  2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.
  , examples=[(QueryCondition(persona=Persona(name='Software Engineer', role_description='Focuses on coding best practices and system design.'), term='microserv

In [64]:
for query, _ in query_distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [65]:
query_distribution

[(SingleHopSpecificQuerySynthesizer(name='single_hop_specifc_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:
  1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.
  2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.
  , examples=[(QueryCondition(persona=Persona(name='소프트웨어 엔지니어', role_description='코딩 모범 사례와 시스템 설계에 집중합니다.'), term='마이크로서비스', query_style='정식', query_length='중

In [71]:
from ragas.testset.persona import default_filter, generate_personas_from_kg

In [68]:
len([node for node in new_kg.nodes if default_filter(node)])

23

In [72]:
persona_list=generate_personas_from_kg(
    llm = generator_llm,
    kg = new_kg,
    num_personas=23
    )

Generating personas:   0%|          | 0/23 [00:00<?, ?it/s]

Generating personas: 100%|██████████| 23/23 [00:03<00:00,  7.53it/s]


In [73]:
persona_list

[Persona(name='Financial Data Scientist', role_description='Focuses on utilizing domain-specific language models to analyze and interpret financial datasets, ensuring compliance and security in the application of natural language processing within the financial sector.'),
 Persona(name='Machine Learning Engineer', role_description='Specializes in setting up and optimizing the training environment for deep learning models, considering key factors like batch size, hardware selection, and hyperparameters.'),
 Persona(name='Financial Data Analyst', role_description='Specializes in the collection and analysis of financial datasets to derive insights for market trends and product understanding.'),
 Persona(name='AI Research Scientist', role_description='Specializes in improving machine learning models through techniques like fine-tuning and matrix decomposition, focusing on advancing NLP and NLG capabilities.'),
 Persona(name='Financial Data Scientist', role_description='Utilizes LLMs to enh

In [75]:
testset_generator = TestsetGenerator(embedding_model=ragas_embeddings, llm=generator_llm, knowledge_graph=new_kg, persona_list=persona_list)
samples = testset_generator.generate(testset_size=20, query_distribution=query_distribution)

Generating Samples: 100%|██████████| 22/22 [00:09<00:00,  2.24it/s]


In [85]:
sample_df = samples.to_pandas()
sample_df

,user_input,reference_contexts,reference,synthesizer_name
0,What evaluation benchmarks are used for Bloomb...,[<표 7> Evaluation Benchmarks of BloombergGPT. ...,BloombergGPT uses several evaluation benchmark...,single_hop_specifc_query_synthesizer
1,"금융 특화 파인튜닝이란 무엇이며, 어떤 성능을 보이나요?","[하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융 특화 파인튜닝 LL...",금융 특화 파인튜닝은 금융 분야의 특정 작업에 높은 성능을 보이는 LLM(대형 언어...,single_hop_specifc_query_synthesizer
2,금융 리서치에서 LLM을 활용하는 방법과 그 필요성에 대해 설명해줄 수 있나요?,[금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생성방법을 가이드하기위...,"금융 리서치에서 LLM을 활용하는 방법은 다양한 금융 데이터셋을 확보하여 학습하고,...",single_hop_specifc_query_synthesizer
3,"FinGPT는 금융 분야에서 어떤 역할을 하며, 보안 및 규정 준수와 관련하여 어떤...",[3.4. 금융 분야를 위한 LLM 구성 3.3.3. 보안 및 규정 준수 금융 분야...,"FinGPT는 금융 분야에서 자연어 처리에 특화된 모델로, 금융 데이터의 민감한 정...",single_hop_specifc_query_synthesizer
4,미래에셋증권은 어떤 서비스를 제공하고 있습니까?,"[수 있다. 예를 들어, 모델은 금융 뉴스, 기업 보고 4.2.4. 자동화된 금융 ...",미래에셋증권은 챗 GPT를 활용해 종목의 시황을 요약하는 서비스 작성을 도입하여 금...,single_hop_specifc_query_synthesizer
5,정천수는 무엇을 의미하나요?,"[사전학습하기 위한 사례에 맞게 파인튜닝을 하게 된다(정천수, 2023d). <그림...",정천수는 사전학습하기 위한 사례에 맞게 파인튜닝을 하게 되는 과정을 나타냅니다.,single_hop_specifc_query_synthesizer
6,자동 응답 시스템이 금융 분야에서 고객 응대 품질을 향상시키는 데 어떤 역할을 할 ...,"[고객 응대 개선측 종합적으로, 본 연구는 LLM을 금융 분야에 면에서는 자동 응답...",자동 응답 시스템은 LLM을 금융 분야에 적용하여 고객 응대의 품질을 향상시키는 데...,single_hop_specifc_query_synthesizer
7,"금융 분야에서 LLM의 파인튜닝은 어떻게 이루어지며, FinGPT와 Bloomber...","[<1-hop>\n\n하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융...",금융 분야에서 LLM의 파인튜닝은 특정 작업에 높은 성능을 보이기 위해 금융 특화된...,multi_hop_abstract_query_synthesizer
8,"금융 분야에서 LLM 파인튜닝이 어떻게 이루어지고 있으며, 어떤 모델들이 사용되고 ...","[<1-hop>\n\n하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융...",금융 분야에서 LLM 파인튜닝은 특정 작업에 높은 성능을 보이기 위해 금융 특화된 ...,multi_hop_abstract_query_synthesizer
9,금융 분야에서 모델 성능 극대화를 위해 하이퍼파라미터 조정은 어떻게 이루어지나요?,[<1-hop>\n\n2) 모델의 용도: 모델의 용도에 따라 적합한 3.2.2. 하...,금융 분야에서 모델 성능 극대화를 위해 하이퍼파라미터 조정은 모델의 용도에 따라 적...,multi_hop_abstract_query_synthesizer


## 테스트셋 체크

In [78]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [82]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [193]:
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser

class QA_check(BaseModel):
    validity: bool
    reference_context_relevance: bool
    qa_appropriateness:bool 
    

parser = PydanticOutputParser(pydantic_object=QA_check)

In [269]:
prompt = """
다음 질문과 답변이 주어진 논문을 기준으로 적절한지 평가하십시오.

다음 항목을 순차적으로 평가해 주세요:

1. 질문의 적절성 (Validity)
- 질문이 논문에 언급된 용어, 표현, 개념에 대해 논문에서 제공하는 정보를 바탕으로 답할 수 있도록 명확히 구성되었나요?
- 논문에서 인용된 사람의 이름 또는 특정 저자명을 단순 인용 표기(예: (홍길동, 2024))임에도 불구하고 단어나 개념처럼 잘못 해석하여 그 의미를 묻는 질문은 부적절합니다.
(예: 논문 인용 표기에서 "(홍길동, 2024)"와 같은 저자 이름을 하나의 개념이나 용어로 잘못 파악하고 질문하는 경우는 잘못된 질문임.)

2. 참조 문맥의 관련성 (Reference Context Relevance)
- 제공된 참조 문맥이 질문과 답변에 관련 있으며, 논문의 실제 내용과 정확히 일치하는가요?

3. 답변의 적합성 (QA Appropriateness)
- 제공된 답변이 논문에서 실제로 제시된 정보를 근거로 질문에 적절히 대응하는가요?

# 질문
{user_input}

# 답변
{reference}

# 참조 문맥
{reference_contexts}

# 포맷
```json
{{"validity": true or false, "reference_context_relevance": true or false, "qa_appropriateness": true or false}}
```

Answer:
"""

In [ ]:
chain = RunnablePassthrough()|PromptTemplate(template=prompt, input_variables=["user_input", "reference", "reference_contexts"])|llm_gemini|parser

In [271]:
check_sample = sample_df.loc[5,:"reference"]
check_sample.to_dict()

{'user_input': '정천수는 무엇을 의미하나요?',
 'reference_contexts': ['사전학습하기 위한 사례에 맞게 파인튜닝을 하게 된다(정천수, 2023d). <그림 3> LLM 사전 학습 절차 99'],
 'reference': '정천수는 사전학습하기 위한 사례에 맞게 파인튜닝을 하게 되는 과정을 나타냅니다.'}

In [272]:
ans_check = chain.invoke(check_sample.to_dict())

In [273]:
ans_check

QA_check(validity=False, reference_context_relevance=True, qa_appropriateness=False)

In [ ]:
import time

In [ ]:
ans_total = []
for sample_start in range(0,sample_df.shape[0],10):
    print(sample_start, sample_start+10)
    ans = chain.batch(list(sample_df.loc[sample_start:sample_start+10,:"reference"].T.to_dict().values()))
    ans_total.extend(ans)
    time.sleep(60.0)

0 10
10 20
20 30


In [ ]:
len(sample_df.loc[sample_start:sample_start+10,:"reference"].T.to_dict().values())

2

In [ ]:
len(ans_total)

24

In [ ]:
import pandas as pd

In [ ]:
pd.concat([sample_df,pd.DataFrame([i.model_dump() for i in ans_total])], axis=1).dropna()

,user_input,reference_contexts,reference,synthesizer_name,validity,reference_context_relevance,qa_appropriateness
0,What evaluation benchmarks are used for Bloomb...,[<표 7> Evaluation Benchmarks of BloombergGPT. ...,BloombergGPT uses several evaluation benchmark...,single_hop_specifc_query_synthesizer,True,True,True
1,"금융 특화 파인튜닝이란 무엇이며, 어떤 성능을 보이나요?","[하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융 특화 파인튜닝 LL...",금융 특화 파인튜닝은 금융 분야의 특정 작업에 높은 성능을 보이는 LLM(대형 언어...,single_hop_specifc_query_synthesizer,True,True,True
2,금융 리서치에서 LLM을 활용하는 방법과 그 필요성에 대해 설명해줄 수 있나요?,[금융 데이터의 특성을 고려한 파인튜닝을 통해 된 데이터셋은 생성방법을 가이드하기위...,"금융 리서치에서 LLM을 활용하는 방법은 다양한 금융 데이터셋을 확보하여 학습하고,...",single_hop_specifc_query_synthesizer,True,True,True
3,"FinGPT는 금융 분야에서 어떤 역할을 하며, 보안 및 규정 준수와 관련하여 어떤...",[3.4. 금융 분야를 위한 LLM 구성 3.3.3. 보안 및 규정 준수 금융 분야...,"FinGPT는 금융 분야에서 자연어 처리에 특화된 모델로, 금융 데이터의 민감한 정...",single_hop_specifc_query_synthesizer,True,True,True
4,미래에셋증권은 어떤 서비스를 제공하고 있습니까?,"[수 있다. 예를 들어, 모델은 금융 뉴스, 기업 보고 4.2.4. 자동화된 금융 ...",미래에셋증권은 챗 GPT를 활용해 종목의 시황을 요약하는 서비스 작성을 도입하여 금...,single_hop_specifc_query_synthesizer,True,True,True
5,정천수는 무엇을 의미하나요?,"[사전학습하기 위한 사례에 맞게 파인튜닝을 하게 된다(정천수, 2023d). <그림...",정천수는 사전학습하기 위한 사례에 맞게 파인튜닝을 하게 되는 과정을 나타냅니다.,single_hop_specifc_query_synthesizer,False,True,False
6,자동 응답 시스템이 금융 분야에서 고객 응대 품질을 향상시키는 데 어떤 역할을 할 ...,"[고객 응대 개선측 종합적으로, 본 연구는 LLM을 금융 분야에 면에서는 자동 응답...",자동 응답 시스템은 LLM을 금융 분야에 적용하여 고객 응대의 품질을 향상시키는 데...,single_hop_specifc_query_synthesizer,True,True,True
7,"금융 분야에서 LLM의 파인튜닝은 어떻게 이루어지며, FinGPT와 Bloomber...","[<1-hop>\n\n하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융...",금융 분야에서 LLM의 파인튜닝은 특정 작업에 높은 성능을 보이기 위해 금융 특화된...,multi_hop_abstract_query_synthesizer,True,True,True
8,"금융 분야에서 LLM 파인튜닝이 어떻게 이루어지고 있으며, 어떤 모델들이 사용되고 ...","[<1-hop>\n\n하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융...",금융 분야에서 LLM 파인튜닝은 특정 작업에 높은 성능을 보이기 위해 금융 특화된 ...,multi_hop_abstract_query_synthesizer,True,True,True
9,금융 분야에서 모델 성능 극대화를 위해 하이퍼파라미터 조정은 어떻게 이루어지나요?,[<1-hop>\n\n2) 모델의 용도: 모델의 용도에 따라 적합한 3.2.2. 하...,금융 분야에서 모델 성능 극대화를 위해 하이퍼파라미터 조정은 모델의 용도에 따라 적...,multi_hop_abstract_query_synthesizer,True,True,True


## RAGAS 라이브러리와 형식 통합

In [221]:
from ragas.prompt.pydantic_prompt import PydanticPrompt

In [260]:
class QA_check_input(BaseModel):
    question: str
    answer: str
    reference_context: t.List[str]

class QA_check_output(BaseModel):
    validity: bool
    reference_context_relevance: bool
    qa_appropriateness:bool 

In [261]:
import typing as t
class QA_checkPrompt(PydanticPrompt[QA_check_input, QA_check_output]):
    instruction: str = (
        "Evaluate whether the following question and answer are appropriate based on the provided paper."
        "Please sequentially assess the following items:"

        "1. Question Validity"
        "- Is the question clearly structured based on the terms, expressions, or concepts mentioned in the paper?"
        "- It is inappropriate if the question misinterprets an author's name or citation (e.g., (Hong, 2024)) as a term or concept."

        "2. Reference Context Relevance"
        "- Does the provided reference context accurately match the actual content of the paper and relate properly to the question and answer?"

        "3. QA Appropriateness"
        "- Does the provided answer appropriately respond to the question based on information explicitly stated in the paper?"
    )
    input_model: t.Type[QA_check_input] = QA_check_input
    output_model: t.Type[QA_check_output] = QA_check_output
    examples: t.List[t.Tuple[QA_check_input, QA_check_output]] = [
        (
            QA_check_input(
                question="What impact do few-shot prompting techniques have on LLM models?",
                answer="Few-shot prompting techniques help improve the question-answering performance of LLM models.",
                reference_context=["Recent LLM models are known to improve question-answering performance using few-shot prompting techniques (Lee, 2023)."],
            ),
            QA_check_output(
                validity=True,
                reference_context_relevance=True,
                qa_appropriateness=True
            )
        ),
        (
            QA_check_input(
                question="What does Kim mean?",
                answer="Kim refers to the fine-tuning method of further training models on specific tasks.",
                reference_context=["Fine-tuning is a methodology to improve performance by further training pre-trained models on specific tasks (Kim, 2022)."],
            ),
            QA_check_output(
                validity=False,
                reference_context_relevance=False,
                qa_appropriateness=False
            )
        )
    ]


In [262]:

introduction_prompt_kor = """"
다음 질문과 답변이 주어진 논문을 기준으로 적절한지 평가하십시오.

다음 항목을 순차적으로 평가해 주세요:

1. 질문의 적절성 (Validity)
- 질문이 논문에 언급된 용어, 표현, 개념에 대해 논문에서 제공하는 정보를 바탕으로 답할 수 있도록 명확히 구성되었나요?
- 논문에서 인용된 사람의 이름 또는 특정 저자명을 단순 인용 표기(예: (홍길동, 2024))임에도 불구하고 단어나 개념처럼 잘못 해석하여 그 의미를 묻는 질문은 부적절합니다.
(예: 논문 인용 표기에서 "(홍길동, 2024)"와 같은 저자 이름을 하나의 개념이나 용어로 잘못 파악하고 질문하는 경우는 잘못된 질문임.)

2. 참조 문맥의 관련성 (Reference Context Relevance)
- 제공된 참조 문맥이 질문과 답변에 관련 있으며, 논문의 실제 내용과 정확히 일치하는가요?

3. 답변의 적합성 (QA Appropriateness)
- 제공된 답변이 논문에서 실제로 제시된 정보를 근거로 질문에 적절히 대응하는가요?" \
"""

example_kor = [
        (
            QA_check_input(
                question="Few-shot 프롬프트 기법이 LLM 모델에 미치는 영향은 무엇인가요?",
                answer="Few-shot 프롬프트 기법은 LLM 모델의 질문-답변 성능을 향상시키는 데 도움을 줍니다.",
                reference_context=["최근 LLM 모델은 Few-shot 프롬프트 기법을 활용하여 질문-답변 성능을 향상시킬 수 있다고 알려졌다(AAA, 2023)."],
            ),
            QA_check_output(
                validity=True,
                reference_context_relevance=True,
                qa_appropriateness=True
            )
        ),
        (
            QA_check_input(
                question="BBB는 무엇을 의미하나요?",
                answer="김철수는 특정 작업에 맞게 모델을 추가 학습하는 파인튜닝 방법을 의미합니다.",
                reference_context=["파인튜닝은 특정 작업에 맞게 사전 학습된 모델을 추가 학습하여 성능을 개선하는 방법론이다(BBB, 2022)."],
            ),
            QA_check_output(
                validity=False,
                reference_context_relevance=False,
                qa_appropriateness=False
            )
        )
    ]

qa_check = QA_checkPrompt()
qa_check.instruction = introduction_prompt_kor
qa_check.examples = example_kor

In [252]:
samples.to_evaluation_dataset()[0]

SingleTurnSample(user_input='What evaluation benchmarks are used for BloombergGPT, and what specific tasks does it measure in the financial domain?', retrieved_contexts=None, reference_contexts=['<표 7> Evaluation Benchmarks of BloombergGPT. Suit Tasks What does it measure? Public Financial Tasks 5 Public datasets in the financial domain Bloomberg Financial Tasks 12 NER and sentiment analysis tasks Big-bench Hard (Suzgun et al., 2022) 23 Reasoning and general NLP tasks Knowledge Assessments 5 Testing closed-book information recall Reading Comprehension 5 Testing open-book tasks Linguistic Tasks 9 Not directly user-facing NLP tasks 109'], response=None, multi_responses=None, reference='BloombergGPT uses several evaluation benchmarks, including Public Financial Tasks, which measure 5 public datasets in the financial domain, and Bloomberg Financial Tasks, which consist of 12 NER and sentiment analysis tasks. Additionally, it includes Big-bench Hard (Suzgun et al., 2022) with 23 reasoning a

In [263]:
def sample2qainput(sample):
    return QA_check_input(
        question=sample.user_input,
        answer=sample.reference,
        reference_context=sample.reference_contexts
    )

In [265]:
sample2qainput(samples.to_evaluation_dataset()[1])

QA_check_input(question='금융 특화 파인튜닝이란 무엇이며, 어떤 성능을 보이나요?', answer='금융 특화 파인튜닝은 금융 분야의 특정 작업에 높은 성능을 보이는 LLM(대형 언어 모델)으로, 금융 분야에 특화된 파인튜닝이 적용된 모델입니다.', reference_context=['하여, 금융 분야의 특정 작업에 높은 성능을 3.4.2. 금융 특화 파인튜닝 LLM 보이고 있다. (LLM fine-tuned for finance) 금융 분야에 특화된 파인튜닝이 적용된 모델 '])

In [276]:
test = await qa_check.generate(data=sample2qainput(samples.to_evaluation_dataset()[5]), llm=generator_llm)

In [277]:
test

QA_check_output(validity=False, reference_context_relevance=True, qa_appropriateness=False)